<a href="https://colab.research.google.com/github/Thuy01234567/AIO/blob/main/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.**văn bản in đậm**

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

# Đường dẫn đến trang web gốc
URL = 'https://vnexpress.net'
#URL = 'https://nhandan.vn/'

def get_links(url):
    links = []
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    # Lấy tất cả đường dẫn trong trang web hiện tại
    urls = set(re.findall(r'(?:http|https|fpt):\/\/(?:[\w_-]+(?:(?:\.[\w_-]+)+))(?:[\w.,@?^=%&=:/~+#-]*[\w@?^=%&/~+#-])',
        str(soup)
    ))

    for url in urls:
        # Chỉ lấy đường dẫn là trang web chứa bài báo
        if url != URL and url.startswith(URL) and not url.startswith(URL + '/tac-gia') and url.endswith('.html'):
            links.append(url)

    return links

# Sử dụng hàm để lấy danh sách các đường dẫn
links = get_links(URL)
for link in links:
    print(link)

https://vnexpress.net/brad-pitt-khong-the-lien-lac-pax-thien-sau-tai-nan-4776363.html
https://vnexpress.net/vdv-trieu-tien-va-han-quoc-selfie-chung-tai-olympic-4776508.html
https://vnexpress.net/chung-khoan-hom-nay-1-8-400-ma-tren-san-hose-giam-diem-4776587.html
https://vnexpress.net/hoi-dong-bao-an-lhq-hop-khan-sau-vu-thu-linh-hamas-bi-am-sat-4776380.html
https://vnexpress.net/mua-lon-gay-sat-lo-chia-cat-nhieu-quoc-lo-phia-bac-4776492.html
https://vnexpress.net/bao-tang-truong-sa-co-kien-truc-huong-ve-bien-dong-4776299.html
https://vnexpress.net/cang-thang-vi-bo-me-chon-ho-dai-hoc-4766103.html
https://vnexpress.net/cuu-thanh-nien-roi-xuong-vuc-deo-prenn-4776413.html
https://vnexpress.net/dien-thoai-con-dao-hai-luoi-tren-chien-truong-ukraine-4774165.html
https://vnexpress.net/nadal-trang-tay-o-olympic-paris-2024-4776348.html
https://vnexpress.net/trung-quoc-len-dan-dau-olympic-2024-4776356.html
https://vnexpress.net/phan-manh-quynh-vo-giup-toi-giu-tien-4774969.html
https://vnexpress.ne

In [2]:
import requests
from bs4 import BeautifulSoup

# Danh sách các chuyên mục
categories = ['thoi-su', 'goc-nhin', 'the-gioi', 'kinh-doanh', 'bat-dong-san',
              'khoa-hoc', 'giai-tri', 'the-thao', 'phap-luat', 'giao-duc',
              'suc-khoe', 'doi-song','du-lich','oto-xe-may','y-kien','tam-su']

article_url = {}

# Lấy 10 trang trong mỗi chuyên mục
for category in categories:
  article_url[category] = []

  #Lấy 10 trang trong mỗi chuyên mục
  for i in range(1, 11):
    url = '{}/{}-p{}'.format(URL,category,str(i))
    article_url[category] += get_links(url)

# Loại bỏ đường dẫn trùng nhau
for category in article_url:
  article_url[category] = list(set(article_url[category]))
  print(category, len(article_url[category]))

# Lấy riêng các đường dẫn trong chuyên mụcgóc nhìn theo từng thư mục con
goc_nhin = ['chinh-tri-chinh-sach','y-te-suc-khoe','kinh-doanh-quan-tri', 'giao-duc-tri-thuc',
            'moi-truong', 'van-hoa-loi-song', 'covid-19']

article_url['goc-nhin'] = []

for sub_cate in goc_nhin:
  url = '{}/{}/{}'.format(URL,'goc-nhin',sub_cate)
  article_url['goc-nhin'] += get_links(url)

article_url['goc-nhin'] = list(set(article_url['goc-nhin']))
print('goc-nhin', len(article_url['goc-nhin']))

thoi-su 313
goc-nhin 103
the-gioi 285
kinh-doanh 287
bat-dong-san 330
khoa-hoc 299
giai-tri 304
the-thao 306
phap-luat 296
giao-duc 316
suc-khoe 306
doi-song 308
du-lich 323
oto-xe-may 319
y-kien 302
tam-su 70
goc-nhin 89


2.

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

def get_content(url):
    # Mở trang bài báo
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    # Lấy phần chứa nội dung bài báo
    div_content = soup.select('.page-detail .container')
    if len(div_content) > 0:
        div_content = div_content[0]

        # Lấy phần mô tả
        description = div_content.find_all('p', {'class': 'description'})
        if len(description) > 0:
            description = description[-1]
            text_description = description.get_text()
            location = description.find('span', {'class': 'location-stamp'})
            if location is not None:
                content = text_description[len(location.get_text()):]
            else:
                content = text_description
        else:
            content = ""

        # Lấy phần chứa nội dung chi tiết bài báo
        detail = div_content.find('article', {'class': 'ck_detail'})
        if detail is not None:
            p_normal = detail.find_all('p', {'class': 'Normal'})
            if len(p_normal) > 0:
                for p in p_normal:
                    p_text = p.get_text()
                    if p_text != '' and not p_text.startswith(('>>')) and p_text[-1] in ('.', '?', '!', ':'):
                        content += ' ' + p.get_text()
    else:
        content = ""

    return re.sub(r'\n','',content)

In [4]:
get_content(article_url['giao-duc'][2])

'Bỏ ôn thi học sinh giỏi quốc gia, Khôi Nguyên tập trung cho kỳ thi tốt nghiệp, đạt trung bình 9,58/10 điểm bài tổ hợp Khoa học tự nhiên, cao nhất toàn quốc.'

In [5]:
import os
import re

# Danh sách tất cả các câu đã làm sạch của tất cả các bài viết
sentence = []
# Chữ cái in hoa của tiếng Việt
uppercase = 'AÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶBCDĐEÉÈẺẼẸÊẾỀỂỄỆFGHIÍÌỈĨỊJKLMNOÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢPQRSTUÚÙỦŨỤƯỨỪỬỮỰVWXYZÝỲỶỸỴZ'

for category in article_url.keys():
    # Tạo folder chuyên mục để chứa các file bài viết
    current_path = 'data/categories/' + category + '/'
    os.makedirs(current_path, exist_ok=True)

    # Lưu các bài viết thành từng file
    count = 0
    for i in range(len(article_url[category])):
      content = get_content(article_url[category][i])
      if content != '':
        count += 1
        f = open('{}/{}_{}.txt'.format(current_path, category, str(count)), 'w', encoding='utf-8')

        # Chèn khoảng cách vào sau các câu chưa chuẩn
        lines = re.sub(r'(?<=[.?!])(?=[\'"`“’]*['+ uppercase + '-])', ' ', content)

        # Tách bài viết thành các câu
        lines = re.split(r'(?<=[.?!])\s+(?=[\'"`“’]*[' + uppercase + '0-9-])', content)


        # Ghi từng câu trong bài viết vào file
        for line in lines:
          f.write(line + '\n')
          sentence.append(line)
        f.close()
    #Số bài viết trong chuyên mục
    print(category, count)
sentence =list(set(sentence))
print('Tổng số câu tách được:', len(sentence))

thoi-su 312
goc-nhin 89
the-gioi 285
kinh-doanh 286
bat-dong-san 326
khoa-hoc 299
giai-tri 304
the-thao 306
phap-luat 296
giao-duc 313
suc-khoe 306
doi-song 280
du-lich 318
oto-xe-may 308
y-kien 302
tam-su 69
Tổng số câu tách được: 4478


In [6]:
import os
import re

with open('/content/data/corpus.txt', 'w', encoding="utf-8") as file_corpus:
    count = 0

    for sent in sentence:
        # Xóa bỏ ký tự thừa
        sent = re.sub(r'\u200b', '', sent)  # Use re.sub() for substitution

        # Ghi câu vào file ngữ liệu
        file_corpus.write(sent.strip() + '\n')

        # Đếm số lượng câu
        count += 1

print("Số lượng câu trong file ngữ liệu:", count)

Số lượng câu trong file ngữ liệu: 4478


In [ ]:
from google.colab import drive
drive.mount('/content/drive')